# Two-point microrheology (example)

Load a CSV of tracer trajectories and compute two-point displacement correlations + derived MSDs using `trajkit.flow.two_point`.

This mirrors the classic Crocker/Mason MATLAB workflow (`twopoint.m` → `msdd` → `calc_G`).

In [ ]:
import numpy as np
import pandas as pd
from trajkit.flow.two_point import (
    compute_two_point_correlation,
    distinct_msd_from_two_point,
    compute_shear_modulus_from_msd,
)
from trajkit.traj.core import TrajectorySet

from pathlib import Path
from huggingface_hub import snapshot_download
from trajkit import load_trajectory_set

In [ ]:



subpath = "data/parquet/experiment_001_2017-08-16/exp001_t027m_r01um_2017-08-16"

local_root = snapshot_download(
    repo_id="m-aban/air-water",
    repo_type="dataset",
    allow_patterns=[f"{subpath}/*"],
    local_dir="hf_cache",
    local_dir_use_symlinks=False,
)

folder = Path(local_root) / subpath
ts = load_trajectory_set(folder)        


In [ ]:
import numpy as np

In [ ]:
# Compute two-point correlations
tau = np.ceil(np.logspace(0,2,10)) # lag time
corr = compute_two_point_correlation(
    ts,
    track_id_col="id",
    time_col="t",
    position_cols=("x", "y"),
    dt_values=tau,      # auto log-spaced lags
    max_dt=100,          # max lag in frames if dt_values=None
    r_min=10,
    r_max=500,
    n_r_bins=20,
    clip_to_shared_frames=True,
)


In [ ]:
from trajkit.flow import compute_two_point_correlation, save_two_point_correlation, load_two_point_correlation

# corr = compute_two_point_correlation(ts, max_dt=50, r_min=0.5, r_max=20.0)
save_two_point_correlation(corr, "results/two_point_corr.npz")

# Later / elsewhere
corr_loaded = load_two_point_correlation("results/two_point_corr.npz")


In [ ]:
from trajkit.flow import compute_two_point_correlation, save_two_point_correlation, load_two_point_correlation

save_two_point_correlation(corr, "results/two_point_corr.npz")

# Later / elsewhere
# corr_loaded = load_two_point_correlation("results/two_point_corr.npz")


In [ ]:
import matplotlib.pyplot as plt
plt.scatter(corr.r, corr.longitudinal[1,:])
plt.xscale('log')
plt.yscale('log')

In [ ]:
# Convert two-point correlations to one-point-like MSDs (analogous to msdd.m)
msd_from_2p = distinct_msd_from_two_point(
    corr,
    r_min=10,
    r_max=500,
    probe_radius=1.0,   # microns
    use_linear_fit=False,
)
msd_from_2p.msd_longitudinal[:5], msd_from_2p.msd_transverse[:5]

In [ ]:
msd_from_2p.msd_longitudinal

In [ ]:
# Compute viscoelastic moduli from MSD (analogous to calc_G.m / Mason-Weitz)
moduli = compute_shear_modulus_from_msd(
    tau=msd_from_2p.dt,
    msd=msd_from_2p.msd_transverse,  # choose L or T
    probe_radius_microns=1.0,
    dim=2,
    temperature_K=298.0,
    clip=0.03,
    smoothing_window=7,
    polyorder=2,
)
moduli.omega[:5], moduli.Gp[:5], moduli.Gpp[:5]

In [ ]:
import os
from huggingface_hub import hf_hub_download
path = hf_hub_download(
    repo_id="m-aban/air-water",
    filename="data/csv/experiment_001_2017-08-16/exp001_t000m_r01um_2017-08-16.csv.gz",
    repo_type="dataset",
)


In [ ]:
import pandas as pd
df = pd.read_csv(path)

In [ ]:
import os
from huggingface_hub import hf_hub_download
pathTraj = hf_hub_download(
    repo_id="m-aban/air-water",
    filename="data/parquet/experiment_001_2017-08-16/exp001_t000m_r01um_2017-08-16/tracks_index.parquet",
    repo_type="dataset",
)
ts = pd.read_parquet(pathTraj)
ts


In [ ]:
from trajkit import TrajectorySet
df = pd.read_parquet("tracks.parquet")
ts = TrajectorySet.from_dataframe(df, dataset_id="air-water")

In [ ]:
import pandas as pd
import numpy as np
from trajkit import TrajectorySet
from trajkit.flow import compute_two_point_correlation

csv_path = "/Users/mehdi/air-water/data/csv/experiment_001_2017-08-16/exp001_t011m_r01um_2017-08-16.csv.gz"  # change if needed
df = pd.read_csv(csv_path)
df = df[(df["t"]<500) & (df["t"]>100)]

ts = TrajectorySet.from_dataframe(
    df,
    dataset_id="air-water-csv",
    track_id_col="id",           # change if your CSV uses e.g. "track_id"
    position_cols=["x", "y"],    # add "z" if 3D
    time_col="t",                # or use frame_col="frame", frame_rate_hz=20.0 if you have frames instead of time
)

tau = np.ceil(np.logspace(0,2,10)) # lag time
corr = compute_two_point_correlation(
    ts,
    track_id_col="id",
    time_col="t",
    position_cols=("x", "y"),
    dt_values=tau,      # auto log-spaced lags
    max_dt=100,          # max lag in frames if dt_values=None
    r_min=10,
    r_max=500,
    n_r_bins=20,
    clip_to_shared_frames=True,
)


In [ ]:
import matplotlib.style as mplstyle
from importlib.resources import files
import matplotlib.pyplot as plt

mplstyle.use(files("trajkit.styles").joinpath("plot.mplstyle"))


In [ ]:

plt.scatter(corr.r,corr.longitudinal[0,:])

In [ ]:
folder = Path(local_root) / "data/parquet/experiment_001_2017-08-16/exp001_t027m_r01um_2017-08-16"
ts = load_trajectory_set(folder)   